## Knowledge distillation 

In [1]:

from torchvision import datasets, transforms
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.utils.prune as prune
import time,os,copy
import random

In [2]:
# to make the note book reproducable
def seed_all(seed):
    if not seed:
        seed = 10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_all(1)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:

class Data(Dataset):
    """
    Define Dataset
    """
    def __init__(self,data):
        self.data = data
        # pd.read_csv('mnist_train_small.csv').iloc[:][1:]
#         np.array(train.iloc[:][1:])
        self.x=np.array(self.data.iloc[:,1:])
        self.y= np.array(self.data.iloc[:,0])
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


In [5]:
# hyper paramters
criterion = nn.CrossEntropyLoss()
epoches=20
bs=32
epoches=50
lr=.001

In [27]:
# Here I used a small verison of the mnsit thet come on google colab by default
train_set=torch.utils.data.DataLoader(Data( pd.read_csv('mnist_train_small.csv').iloc[:][1:]), batch_size=bs)
test=torch.utils.data.DataLoader(Data(pd.read_csv('mnist_test.csv').iloc[:][1:]), batch_size=bs,shuffle=True)

In [8]:
class LeNet(nn.Module):
    """ Lenet model """
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
def train (model,teacher,train_set,loss_criterion,soft=False):
    optimizer = torch.optim.SGD(model.parameters(), lr=.001)
    
    model.train()
    model.to(device)
    
    if soft:
      teacher.eval()
      kl_div_loss = nn.KLDivLoss(log_target=True)
      soft_targets_weight: float = 100.
      temperature: float = 10
      label_loss_weight: float = 0.5;

    for epoch in tqdm(range(100)):

        
        for images, labels in train_set: 

            images=images.reshape(-1,1,28,28).float()
            images=images.to(device)
            outputs = model(images) 

            labels = labels.to(device)        
            # Forward pass
            loss = loss_criterion(outputs, labels)
            
            # this is he key step
            # is the training is with distillation soft is True
            # the loss will be adjusted further
            if soft:
                # the predictions of the teacher
                large_logits = teacher(images)
                
                # the predictions of the teacher are magnified with a temprature
                soft_targets = nn.functional.log_softmax(large_logits /temperature, dim=-1)
                
                # do softmax
                soft_prob = nn.functional.log_softmax(outputs / temperature, dim=-1)
                
                # loss for soft 
                soft_targets_loss = kl_div_loss(soft_prob, soft_targets)

                # label_loss = loss_func(outputs, labels)
                loss = soft_targets_weight * soft_targets_loss + label_loss_weight * loss


            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print( 'loss : ', loss.item())

In [10]:
def evaluate_model(model,test):
    """ This is the conventional loss function"""
    model.eval()
    model.to(device)
    n_samples = len(test)
    with torch.no_grad():
      n_correct = 0
      for images, labels in test:
          images=images.reshape(-1,1,28,28).float()
          # images = images.reshape(-1, 28*28).to(device)

          labels = labels.to(device);        
          images = images.to(device)
          
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          n_samples += labels.size(0)
          n_correct += (predicted == labels).sum().item()

      return 100.0 * n_correct / n_samples

In [11]:
teacher=LeNet().to(device)
teacher.load_state_dict(torch.load('Lenet95', map_location=device))

<All keys matched successfully>

In [12]:
class smaller_LeNet(nn.Module):
    """
    Define a smaller network
    """
    def __init__(self):
        super(smaller_LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 10) 

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = self.fc1(x)
        return x

In [19]:
# to make sure the models start from equal 

student=smaller_LeNet()

scratch=copy.deepcopy(student)

In [20]:
# keep the models initial accuracy  
teacher_acc=evaluate_model(teacher,test)
std_acc=evaluate_model(student,test)
scr_acc=evaluate_model(scratch,test)

In [21]:
std_acc,scr_acc, teacher_acc

(4.597032295606634, 4.597032295606634, 95.75210939773058)

In [22]:
# train (model,teacher,train_set,loss_criterion,soft=False):
train(scratch,teacher, train_set,criterion)

train(student,teacher, train_set,criterion,soft=True)

  1%|█                                                                                                      | 1/100 [00:03<05:00,  3.03s/it]

loss :  0.3419564366340637


  2%|██                                                                                                     | 2/100 [00:05<04:49,  2.95s/it]

loss :  0.26173868775367737


  3%|███                                                                                                    | 3/100 [00:08<04:42,  2.91s/it]

...

loss :  0.003942582756280899


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [12:56<00:07,  7.30s/it]

loss :  0.0035095971543341875


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [13:03<00:00,  7.84s/it]

loss :  0.004373255651444197


In [23]:
evaluate_model(student,test),evaluate_model(scratch,test)

(95.13141305401997, 94.21006691882455)

### The  student model outperforms the scratch model